In [ ]:
import json
import requests
import pandas as pd

# adjust the query variable to change your search terms

query=requests.get("https://www.digitalcommonwealth.org/search.json?f%5Bgenre_basic_ssim%5D%5B%5D=Cards&f%5Bsubject_facet_ssim%5D%5B%5D=Cities+%26+towns&f%5Bsubject_facet_ssim%5D%5B%5D=Beaches&f%5Bsubject_geographic_sim%5D%5B%5D=United+States&per_page=20")
response = json.dumps(query.json(), indent=2)
data = json.loads(response)

this = (data['links']['self'])
next = (data['links']['next'])
final = (data['links']['last'])

In [ ]:
# set pandas dataframe column names

columnNames=[]
for record in data['data']:    
    for field in record['attributes']:
        if field not in columnNames:
            columnNames.append(field)

out=pd.DataFrame(columns=columnNames)

In [ ]:
# walk through search results in JSON format using DC JSON API

while True:

    # get THIS page and set desired metadata fields

    print("This page: "+this)
    print("Next page: "+next)
    print("Last page: "+final)
    for record in data['data']:
        print(f"Processing object ID: {id}")
        id=record['id']
        q=requests.get(f'https://www.digitalcommonwealth.org/search/{id}.json')
        resp=json.dumps(q.json())
        d=json.loads(resp)
        i=d['data']['attributes']

        # print desired THIS metadata fields

        df=pd.DataFrame([i])
        out=pd.concat([out, df])

    # get NEXT page

    query=requests.get(next)
    response=json.dumps(query.json(), indent=2)
    data = json.loads(response)
    
    # reset all THIS variables to NEXT
    # but save & break if you're at the end

    if this == final:
        out.to_csv("out.csv")
        break
    else:
        this = (data['links']['self'])
        if len(data['links']) > 3:
            next = (data['links']['next'])
        final = (data['links']['last'])
    
    out.to_csv("out.csv")